In [ ]:
"""This is a minimal example of using Tianshou with MARL to train agents.

Author: Will (https://github.com/WillDudley)

Python version used: 3.8.10

Requirements:
pettingzoo == 1.22.0
git+https://github.com/thu-ml/tianshou
"""

import os
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.classic import tictactoe_v3


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_learn is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    return PettingZooEnv(tictactoe_v3.env())


if __name__ == "__main__":
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(10)])
    test_envs = DummyVectorEnv([_get_env for _ in range(10)])

    # seed
    seed = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()

    # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(20_000, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=64 * 10)  # batch size * training_num

    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):
        model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
        os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 0.6

    def train_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.1)

    def test_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.05)

    def reward_metric(rews):
        return rews[:, 1]

    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=50,
        step_per_epoch=1000,
        step_per_collect=50,
        episode_per_test=10,
        batch_size=64,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=0.1,
        test_in_train=False,
        reward_metric=reward_metric,
    )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[1]])")

In [2]:
performance_benchmark(env)

Starting performance benchmark
1305.9148757154892 turns per second
130.59148757154892 cycles per second
Finished performance benchmark


In [1]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
from pettingzoo.test import performance_benchmark
import cProfile
cProfile.run("performance_benchmark(env)")

/home/yuan/miniconda3/envs/man_pypy/lib/pypy3.9/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 2.0.0.dev0+60.g174dfae62)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Starting performance benchmark
659.1720242756144 turns per second
65.91720242756143 cycles per second
Finished performance benchmark
         3272124 function calls (3272116 primitive calls) in 4.705 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3301    0.019    0.000    0.105    0.000 <frozen importlib._bootstrap>:1002(_find_and_load)
     3301    0.005    0.000    0.119    0.000 <frozen importlib._bootstrap>:1018(_gcd_import)
     3301    0.022    0.000    0.147    0.000 <frozen importlib._bootstrap>:1097(__import__)
     6602    0.016    0.000    0.017    0.000 <frozen importlib._bootstrap>:112(release)
     3301    0.006    0.000    0.006    0.000 <frozen importlib._bootstrap>:152(__init__)
     3301    0.006    0.000    0.035    0.000 <frozen importlib._bootstrap>:156(__enter__)
     3301    0.003    0.000    0.012    0.000 <frozen importlib._bootstrap>:160(__exit__)
     6602    0.017    0.000    0.021    0.000

In [2]:
%conda install scipy

Solving environment: done

## Package Plan ##

  environment location: /home/yuan/miniconda3/envs/man_pypy

  added / updated specs:
    - scipy


The following NEW packages will be INSTALLED:

  libgfortran-ng     pkgs/main/linux-64::libgfortran-ng-11.2.0-h00389a5_1 
  libgfortran5       pkgs/main/linux-64::libgfortran5-11.2.0-h1234567_1 
  scipy              pkgs/main/linux-64::scipy-1.9.3-py39hf6e8229_2 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
ge = env.get_job_next_step()

In [5]:
job,dob = next(ge)
job.show()

+-------------------------------------------------------------------------------------------------------------------------------------+
|                                                               Job Info                                                              |
| Job Id | Res Vector | Job Len | Priority |       Budget       | Restrict Machines | Enter Time | Start Time | Finish Time | Utility |
|   2    | [11.  4.]  |    2    |    3     | 136.50789973353466 |     [6 2 7 4]     |     0      |     -1     |      -1     |    0    |
Job Vector
[array([11.,  4.]), array([11.,  4.])]
